## Setup environment

Run the following cells to initialize the notebook instance. Get the SageMaker region, bucket and execution role used to run this notebook.


In [161]:
%pip install --upgrade pip

In [160]:
#libs need to run pipeline from notebook
%pip install sagemaker 
%pip install numpy==1.24.4
%pip install boto3

In [162]:
import boto3
import sagemaker
from ml_pipelines._utils import get_pipeline_driver, convert_struct, get_pipeline_custom_tags
import json


region = boto3.Session().region_name
# This instantiates a SageMaker session that we will be operating in.

session = sagemaker.Session()

# This object represents the IAM role that we are assigned.
role = sagemaker.get_execution_role()
print("Execution role ",role)
bucket = session.default_bucket()
print("Bucket ", bucket)


Exectuion role  arn:aws:iam::596365734174:role/SC-596365734174-pp-cyj5uq-smrolessagemakerstudioro-CF4WRIWNDOFI
Bucket  sagemaker-us-east-1-596365734174


## Prepare pipeline

Configure pipeline_name and description

```
pipeline_name = 'pipeline-local-fs-integration'
description="Testing with local fs"
```

Configure tags, set your sagemaker project arn

```
tags = '[{"Key":"sagemaker:project-arn", "Value":"arn:aws:sagemaker:us-east-1:596365734174:project/bank-loan-default"}]'
```

In [163]:

module_name= "ml_pipelines.training.pipeline"
pipeline_name = 'pipeline-local-fs-integration'
description="Testing with local fs"
kwargs ='{"region":\"'+region+'","pipeline_name":\"'+pipeline_name+'","default_bucket":\"'+bucket+'"}'

#change your project arn
tags = '[{"Key":"sagemaker:project-arn", "Value":"arn:aws:sagemaker:us-east-1:596365734174:project/bank-loan-default"}]'

pipeline = get_pipeline_driver(module_name , kwargs)
print("###### Creating/updating a SageMaker Pipeline with the following definition:")
parsed = json.loads(pipeline.definition())
print(json.dumps(parsed, indent=2, sort_keys=True))

all_tags = get_pipeline_custom_tags(module_name, kwargs, convert_struct(tags))

upsert_response = pipeline.upsert(role_arn=role, description=description, tags=all_tags)

upsert_response = pipeline.upsert(
    role_arn=role, description=description
)  # , tags=tags) # Removing tag momentously
print("\n###### Created/Updated SageMaker Pipeline: Response received:")
print(upsert_response)

execution = pipeline.start()
print(f"\n###### Execution started with PipelineExecutionArn: {execution.arn}")

#         TODO removing wait time as training can take some time
print("Waiting for the execution to finish...")
execution.wait()
print("\n#####Execution completed. Execution step details:")

print(execution.list_steps())

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


Pipeline role ===> arn:aws:iam::596365734174:role/SC-596365734174-pp-cyj5uq-smrolessagemakerstudioro-CF4WRIWNDOFI
###### Creating/updating a SageMaker Pipeline with the following definition:


{
  "Metadata": {},
  "Parameters": [
    {
      "DefaultValue": "ml.m5.large",
      "Name": "ProcessingInstanceType",
      "Type": "String"
    },
    {
      "DefaultValue": 1,
      "Name": "ProcessingInstanceCount",
      "Type": "Integer"
    },
    {
      "DefaultValue": "ml.m4.xlarge",
      "Name": "TrainingInstanceType",
      "Type": "String"
    },
    {
      "DefaultValue": 1,
      "Name": "TrainingInstanceCount",
      "Type": "Integer"
    },
    {
      "DefaultValue": "PendingManualApproval",
      "Name": "ModelApprovalStatus",
      "Type": "String"
    }
  ],
  "PipelineExperimentConfig": {
    "ExperimentName": {
      "Get": "Execution.PipelineName"
    },
    "TrialName": {
      "Get": "Execution.PipelineExecutionId"
    }
  },
  "Steps": [
    {
      "Arguments": {
        "AppSpecification": {
          "ContainerEntrypoint": [
            "python3",
            "/opt/ml/processing/input/code/pre_processing.py"
          ],
          "ImageUri": "6833136


###### Created/Updated SageMaker Pipeline: Response received:
{'PipelineArn': 'arn:aws:sagemaker:us-east-1:596365734174:pipeline/module4-bank-pipeline-local-fs-integration', 'ResponseMetadata': {'RequestId': 'a57803fe-e7ec-4b64-ae13-1de429449f09', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'a57803fe-e7ec-4b64-ae13-1de429449f09', 'content-type': 'application/x-amz-json-1.1', 'content-length': '110', 'date': 'Fri, 27 Oct 2023 12:27:58 GMT'}, 'RetryAttempts': 0}}

###### Execution started with PipelineExecutionArn: arn:aws:sagemaker:us-east-1:596365734174:pipeline/module4-bank-pipeline-local-fs-integration/execution/bkd5tptmhrcv
Waiting for the execution to finish...

#####Execution completed. Execution step details:
[{'StepName': 'RegisterModel-RegisterModel', 'StartTime': datetime.datetime(2023, 10, 27, 12, 28, 2, 854000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2023, 10, 27, 12, 28, 3, 967000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'AttemptCount': 0, 'Met